## Retrieve Data From Wikipedia

This block generates our corpus. That is -
1. download text from Wikipedia pages
2. creates clean text (with not punctuation) = X
3. learn the correct labeling of each word = Y , where $y \in Y$ is $y \in $ {'None','Comma','dot'}

After running this block the corpus = X will be in the file 'pos_to_tag.txt' in the sampe folder of this JN

In [2]:
import os 
import sys
sys.path.append('..\GenerationPipe')
import NLP_HEBPUNCT_GP_generator as gen
import NLP_HEBPUNCT_GP_pre_processing as preproc
import subprocess
from subprocess import check_output

In [3]:


targetUrlList = ["https://he.wikipedia.org/wiki/%D7%A7%D7%A4%D7%99%D7%98%D7%9C%D7%99%D7%96%D7%9D",
                 "https://he.wikipedia.org/wiki/%D7%A7%D7%A8%D7%9C_%D7%9E%D7%A8%D7%A7%D7%A1",
                 "https://he.wikipedia.org/wiki/%D7%90%D7%99%D7%9C%D7%9F_%D7%A8%D7%9E%D7%95%D7%9F",
                 "https://he.wikipedia.org/wiki/%D7%90%D7%9C%D7%91%D7%A8%D7%98_%D7%90%D7%99%D7%99%D7%A0%D7%A9%D7%98%D7%99%D7%99%D7%9F",
                 "https://he.wikipedia.org/wiki/%D7%90%D7%94%D7%91%D7%94",
                 "https://he.wikipedia.org/wiki/%D7%9E%D7%A6%D7%99%D7%90%D7%95%D7%AA",
                 "https://he.wikipedia.org/wiki/%D7%A4%D7%99%D7%9C%D7%95%D7%A1%D7%95%D7%A4%D7%99%D7%94_%D7%A9%D7%9C_%D7%94%D7%9E%D7%93%D7%A2",
                 "https://he.wikipedia.org/wiki/%D7%9B%D7%91%D7%99%D7%93%D7%94",
                 "https://he.wikipedia.org/wiki/%D7%94%D7%9E%D7%90%D7%94_%D7%94-20",
                 "https://he.wikipedia.org/wiki/%D7%9E%D7%96%D7%A8%D7%97_%D7%90%D7%99%D7%A8%D7%95%D7%A4%D7%94",
                 "https://he.wikipedia.org/wiki/%D7%97%D7%95%D7%A7_%D7%94%D7%94%D7%A1%D7%93%D7%A8%D7%99%D7%9D",
                 "https://he.wikipedia.org/wiki/%D7%94%D7%94%D7%A1%D7%AA%D7%93%D7%A8%D7%95%D7%AA_%D7%94%D7%9B%D7%9C%D7%9C%D7%99%D7%AA_%D7%A9%D7%9C_%D7%94%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%91%D7%90%D7%A8%D7%A5_%D7%99%D7%A9%D7%A8%D7%90%D7%9C",
                 "https://he.wikipedia.org/wiki/%D7%90%D7%93%D7%A8%D7%99%D7%9B%D7%9C%D7%95%D7%AA_%D7%92%D7%95%D7%AA%D7%99%D7%AA",
                 "https://he.wikipedia.org/wiki/%D7%9B%D7%A0%D7%A1%D7%99%D7%99%D7%AA_%D7%94%D7%91%D7%A9%D7%95%D7%A8%D7%94",
                 "https://he.wikipedia.org/wiki/%D7%94%D7%9E%D7%97%D7%9C%D7%95%D7%A7%D7%AA_%D7%A2%D7%9C_%D7%94%D7%98%D7%A7%D7%A1%D7%99%D7%9D_%D7%94%D7%A1%D7%99%D7%A0%D7%99%D7%99%D7%9D"]



def word2features(word,i):
    
    features = {
        'bias': 1.0,
        'word[:1]': word[:1],
        'word[:2]': word[:2],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit(),
        'index': i

    }
    
    return features

def words2features(words_list):
    vectors = []
    for i in range(len(words_list)):
        vectors.append(word2features(words_list[i],i))
    return vectors

def texts_to_files(lst):
    file = open("words_to_pos.txt","w") 
    for x in lst:
        s = x[:x.index("<XYDEL>")]
        file.write(s + " ") 
    file.close() 

def generate_dataset(url_list):
    gen.generate(url_list,"train_data")
    out = gen.getStringFromDataFile("train_data")
    out = out.split("<DATADEL>")
    texts_to_files(out)
    data = []
    for x in out:
        data.append(x.split("<XYDEL>"))
    dataset=[]
    for d in data:
        temp = []
        temp.append(d[0].split(" "))
        temp.append(d[1].split(" "))
        dataset.append(temp)
    for i in range(len(dataset)):
        dataset[i][0] = words2features(dataset[i][0])
    return dataset

data = generate_dataset(targetUrlList)
x_train = [x[0] for x in data]
y_train = [x[1] for x in data]

print("-----------------------------------------------------")
print(x_train[0])
print("-----------------------------------------------------")
print(y_train[0])

adding to data set URL: https://he.wikipedia.org/wiki/%D7%A7%D7%A4%D7%99%D7%98%D7%9C%D7%99%D7%96%D7%9D


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Administrator\\Documents\\University\\Semester_C\\NLP\\Project\\NLP_Final_Project\\NLP_Final_Project\\data\\train_data_indx.txt'

## POS tagging the text

The POS of a word considered to be a useful feature for labeling. As we don't have the true POS of each word we create the POS ourselfs. We do this by Using The Hebrew POS tagger Of Meni as described [here](https://www.cs.bgu.ac.il/~elhadad/nlpproj/LDAforHebrew.html)

Run this block **only if you downloaded and installed Meni's Hebrew Tagger**

In [4]:
# Right now the call only worked from within the directory lemlda. maybe saving the directory to path will help
sys.path.append('C:\Users\Administrator\Documents\University\Semester_C\NLP\Project\lemlda\tagger')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-4-b4b871101d64>, line 2)

In [ ]:
#Here we set our variables the tagger will tag every file in 'corpus' and the output will be in the 'corpus_out' file
tag = "C://Users//Administrator//Documents//University//Semester_C//NLP//Project//lemlda//tagger//tag.bat"
corpus = "C://Users//Administrator//Documents//University//Semester_C//NLP//Project//Corpus"
corpus_out = "C://Users//Administrator//Documents//University//Semester_C//NLP//Project//Corpus_out//tags.txt"

subprocess.call([tag,corpus,corpus_out])

cleaning data - After running this cell, tags will be an array of pairs [word, tag] for every word in corpus

In [ ]:
tagger = open(corpus_out, encoding="utf-8").readlines()
toke_tagger = [tag.split('\t') for tag in tagger]
tags = [[tag[0].replace(' ',''),tag[1].replace('\n','')] for tag in toke_tagger if len(tag)>1]

print(tags[:10]) # to see the structure of output

## Third part - Moshe1 take it from here

In [ ]:
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer


import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(x_train, y_train)



In [ ]:
urlList = ["https://he.wikipedia.org/wiki/%D7%A4%D7%99%D7%9C%D7%95%D7%A1%D7%95%D7%A4%D7%99%D7%94_%D7%A9%D7%9C_%D7%94%D7%9E%D7%93%D7%A2"]
data = generate_dataset(urlList)

x_test = [x[0] for x in data]
y_test = [x[1] for x in data]
y_pred = crf.predict(x_test)
print("test set size: " + str(len(x_test)))
print("---------------------------------------------------")
for i in range(len(y_test[0])):
    print(str(i) + " -  truth: " + y_test[0][i] + " prediction: " + y_pred[0][i])
print("---------------------------------------------------")
for i in range(len(y_test[1])):
    print(str(i) + " -  truth: " + y_test[1][i] + " prediction: " + y_pred[1][i])
print("---------------------------------------------------")
for i in range(len(y_test[2])):
    print(str(i) + " -  truth: " + y_test[2][i] + " prediction: " + y_pred[2][i])

In [ ]:
labels = list(crf.classes_)
labels.remove('none')
y_pred = crf.predict(x_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)